In [ ]:
!pip install wandb -qU
!pip install umap-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.8/258.8 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.9/90.9 kB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 5.0 MB/s eta 0:00:00
  Created wheel for umap-learn: filename=umap_learn-0.5.5-py3-none-any.whl size=86832 sha256=a1c5823e86be7871733fc1afb551e41b18e454a1a2f07ce9df56b54a0610344a
  Stored in directory: /root/.cache/pip/wheels/3a/70/07/428d2b58660a1a3b431db59b806a10da736612ebbc66c1bcc5
Successfully built umap-learn


In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, davies_bouldin_score, adjusted_rand_score
from sklearn.preprocessing import normalize
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn import metrics

from google.colab import drive

import numpy as np
import pandas as pd
import time
import os   # This module is used for interacting with the operating system. It provides a way to work with files and directories.
import numba

import tensorflow as tf
import wandb
from wandb.keras import WandbMetricsLogger, WandbModelCheckpoint

import matplotlib.pyplot as plt
import seaborn as sns

import umap

In [ ]:
#wandb.login()

In [ ]:
# --- Load data and setup directories ---
drive.mount('/content/gdrive')
!unzip gdrive/MyDrive/TCGA-BRCA_1079.zip

Mounted at /content/gdrive
Archive:  gdrive/MyDrive/TCGA-BRCA_1079.zip
  inflating: TCGA-BRCA_1079.Xena_TCGA_PanCan.annotation_v6.tsv  
  inflating: TCGA-BRCA_1079_17Kgenes.Xena_TCGA_PanCan.log2_exprs_z_v6.tsv  
  inflating: TCGA-BRCA_1079_17Kgenes.Xena_TCGA_PanCan.subtypes_and_signatures_v6.tsv  


In [ ]:
# --- Prepare labels ---
subtype_labels_frame = pd.read_csv("TCGA-BRCA_1079_17Kgenes.Xena_TCGA_PanCan.subtypes_and_signatures_v6.tsv", sep="\t")
subtype_label_dataset = subtype_labels_frame.values
print ("Shape of subtype label dataset: ", subtype_label_dataset.shape)
print(subtype_label_dataset)
subtype_labels = subtype_label_dataset[:,1]
print ("Shape of subtype labels: ", subtype_labels.shape)
print(subtype_labels)

subtype_label_set = set(subtype_labels)
print(subtype_label_set)

subtype_labels = [0 if item == "Basal" else item for item in subtype_labels]
subtype_labels = [1 if item == "Her2" else item for item in subtype_labels]
subtype_labels = [2 if item == "LumA" else item for item in subtype_labels]
subtype_labels = [3 if item == "LumB" else item for item in subtype_labels]
subtype_labels = [4 if item == "Normal" else item for item in subtype_labels]
subtype_labels = np.array(subtype_labels)
print(subtype_labels.shape)
print(subtype_labels)

Shape of subtype label dataset:  (1079, 34)
[['TCGA-3C-AAAU-01' 'LumB' 0 ... 2 1 0]
 ['TCGA-3C-AALI-01' 'Her2' 0 ... 3 0 0]
 ['TCGA-3C-AALJ-01' 'LumB' 0 ... 3 0 0]
 ...
 ['TCGA-XX-A89A-01' 'LumA' 0 ... 1 0 0]
 ['TCGA-Z7-A8R5-01' 'LumA' 0 ... 1 0 0]
 ['TCGA-Z7-A8R6-01' 'LumB' 0 ... 3 0 0]]
Shape of subtype labels:  (1079,)
['LumB' 'Her2' 'LumB' ... 'LumA' 'LumA' 'LumB']
{'Her2', 'Basal', 'Normal', 'LumA', 'LumB'}
(1079,)
[3 1 3 ... 2 2 3]


In [ ]:
# --- Prepare input data ---
input_data_frame = pd.read_csv("TCGA-BRCA_1079_17Kgenes.Xena_TCGA_PanCan.log2_exprs_z_v6.tsv", sep="\t")   # read .tsv file into memory
input_data = input_data_frame.values  # retrieve values as numpy array
print ("Shape of input data: ", input_data.shape) # rows are genes and columns are samples
print(input_data)

input_data = np.transpose(input_data) # rows are samples and columns are genes
input_data = np.delete(input_data, (0), axis=0) # delete first row of the data matrix
print ("Shape of prepared input data: ", input_data.shape)
print(input_data)

input_data = np.asarray(input_data).astype("float32") # Every character/number encoded with 32 Bit
print("--- Converted to Float32 ---")
print(input_data)

data = input_data
std_scaler = StandardScaler() # Standardize the dataset features so that each feature has mean=0 and standard_deviation=1
scaled_embeddings = std_scaler.fit_transform(data)

reducer = umap.UMAP(n_components=10)
projected_embeddings = reducer.fit_transform(scaled_embeddings)
print("Embeddings shape: ", projected_embeddings.shape)

np.savetxt("UMAP_EM.txt", projected_embeddings)

# --- initialize a new run (<=> single unit of computation) ---
run = wandb.init(project="PCA-projected-BRCA-clustering", # Set the project where this run will be logged
                 name="10_UMAP_new_subtype_comparison", # We pass a run name otherwise it’ll be randomly assigned
                 # Track hyperparameters and run metadata
                 config={
                     "dataset": "BRCA_pca_projected",
                     "number_of_pca": 10,
                     "clustering_algorithm": "k-Means",
                     "centroid_initializations": 20,
                     "performance_aggregation": "mean of all initializations"
                 })

for typenum in range(2, 7, 1):
    all_silhouette = []
    all_DBI = []
    all_adj_rand = []
    for i in range(20):
        clf = KMeans(n_clusters=typenum)
        clf.fit(projected_embeddings)
        labels = clf.labels_
        silhouetteScore = silhouette_score(projected_embeddings, labels, metric='euclidean')
        all_silhouette.append(silhouetteScore)
        davies_bouldinScore = davies_bouldin_score(projected_embeddings, labels)
        all_DBI.append(davies_bouldinScore)
        adj_rand = adjusted_rand_score(subtype_labels, labels)
        all_adj_rand.append(adj_rand)
    avg_silhouette = np.mean(all_silhouette)
    std_silhouette = np.std(all_silhouette)
    avg_DBI = np.mean(all_DBI)
    std_DBI = np.std(all_DBI)
    avg_adj_rand = np.mean(all_adj_rand)
    std_adj_rand = np.std(all_adj_rand)
    print("Silhouette: ", avg_silhouette)
    print("DBI: ", avg_DBI)
    print("Adjusted Rand: ", avg_adj_rand)
    # Log performance metrics to W&B
    wandb.log({
              "mean_silhouette_score_k_equals_{k}".format(k=typenum): avg_silhouette,
              "mean_davis_bouldin_score_k_equals_{k}".format(k=typenum): avg_DBI,
              "std_silhouette_score_k_equals_{k}".format(k=typenum): std_silhouette,
              "std_davis_bouldin_score_k_equals_{k}".format(k=typenum): std_DBI
          })
    if typenum==5:
      wandb.log({
          "mean_adj_rand_index_k_equals_{k}".format(k=typenum): avg_adj_rand,
          "std_adj_rand_index_k_equals_{k}".format(k=typenum): std_adj_rand
      })

# Mark the run as finished
wandb.finish()

Shape of input data:  (17162, 1080)
[['A1BG' 0.1719444908272692 0.7300083449184781 ... 0.614468050540426
  1.3458846686508648 0.7186659277414379]
 ['A1CF' -0.814924951368837 2.386837959210663 ... 3.016626468680079
  -0.814924951368837 0.2356343444931301]
 ['A2M' -0.946474342215696 -0.5321448682087864 ... 0.5601782905367024
  1.512226893819039 -0.6446442624050204]
 ...
 ['DHCR7' 0.9104445032812696 0.3842357893348481 ... 0.7718469491250562
  -0.4217551872682497 -0.970509103357415]
 ['TMEM45B' -1.9635765960806173 1.268061022702539 ... -0.3477150909879061
  0.4043269887748967 -0.5444141861092432]
 ['GPR160' 0.6427457224400778 1.759067645972089 ... 0.3169998035848742
  -0.7956005603903231 -0.1338926768376846]]
Shape of prepared input data:  (1079, 17162)
[[0.1719444908272692 -0.814924951368837 -0.946474342215696 ...
  0.9104445032812696 -1.9635765960806173 0.6427457224400778]
 [0.7300083449184781 2.386837959210663 -0.5321448682087864 ...
  0.3842357893348481 1.268061022702539 1.759067645972

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:8

Silhouette:  0.48614946
DBI:  0.6595564682718817
Adjusted Rand:  0.27646096386537805


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:8

Silhouette:  0.432975
DBI:  0.8514389389962357
Adjusted Rand:  0.24235914964375538


...

Silhouette:  0.38395834
DBI:  1.0009588047904618
Adjusted Rand:  0.2329845543863344


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:8

Silhouette:  0.36057714
DBI:  1.0293341359431698
Adjusted Rand:  0.23570561068919987


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:8

Silhouette:  0.3322205
DBI:  1.134042860728154
Adjusted Rand:  0.21416112719639918
